In [1]:
!pip install openai num2words matplotlib plotly scipy scikit-learn pandas tiktoken


     |████████████████████████████████| 125 kB 5.8 MB/s eta 0:00:01
     |████████████████████████████████| 1.7 MB 56.2 MB/s eta 0:00:01
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13704 sha256=5146c86d532800f00503a14b6c04c7db5ba73a73a78150e31f6ec6dc5b0e4a0b
  Stored in directory: /home/azureuser/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt


In [27]:
import openai
import os
import re
import requests
import sys
from num2words import num2words
import os
import pandas as pd
import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity
import tiktoken

API_KEY = <YOUR_API_KEY>
RESOURCE_ENDPOINT = "https://<YOUR_AZURE_OPENAI_NAME>.openai.azure.com/"

openai.api_type = "azure"
openai.api_key = API_KEY
openai.api_base = RESOURCE_ENDPOINT
openai.api_version = "2022-12-01"

url = openai.api_base + "/openai/deployments?api-version=2022-12-01" 

r = requests.get(url, headers={"api-key": API_KEY})

print(r.text)

{
  "data": [
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "text-davinci-003",
      "owner": "organization-owner",
      "id": "text-davinci-003",
      "status": "succeeded",
      "created_at": 1678106653,
      "updated_at": 1678106653,
      "object": "deployment"
    },
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "text-embedding-ada-002",
      "owner": "organization-owner",
      "id": "text-embedding-ada-002",
      "status": "succeeded",
      "created_at": 1678106668,
      "updated_at": 1678106668,
      "object": "deployment"
    },
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "gpt-35-turbo",
      "owner": "organization-owner",
      "id": "damla-gpt-35",
      "status": "succeeded",
      "created_at": 1678436328,
      "updated_at": 1678436328,
      "object": "deployment"
    },
    {
      "scale_settings": {
        "scale_type": "standa

In [28]:
df=pd.read_csv(os.path.join(os.getcwd(),'./Hotel_Reviews.csv')) 
df

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515733,Wurzbachgasse 21 15 Rudolfsheim F nfhaus 1150 ...,168,8/30/2015,8.1,Atlantis Hotel Vienna,Kuwait,no trolly or staff to help you take the lugga...,14,2823,location,2,8,7.0,"[' Leisure trip ', ' Family with older childre...",704 day,48.203745,16.335677
515734,Wurzbachgasse 21 15 Rudolfsheim F nfhaus 1150 ...,168,8/22/2015,8.1,Atlantis Hotel Vienna,Estonia,The hotel looks like 3 but surely not 4,11,2823,Breakfast was ok and we got earlier check in,11,12,5.8,"[' Leisure trip ', ' Family with young childre...",712 day,48.203745,16.335677
515735,Wurzbachgasse 21 15 Rudolfsheim F nfhaus 1150 ...,168,8/19/2015,8.1,Atlantis Hotel Vienna,Egypt,The ac was useless It was a hot week in vienn...,19,2823,No Positive,0,3,2.5,"[' Leisure trip ', ' Family with older childre...",715 day,48.203745,16.335677
515736,Wurzbachgasse 21 15 Rudolfsheim F nfhaus 1150 ...,168,8/17/2015,8.1,Atlantis Hotel Vienna,Mexico,No Negative,0,2823,The rooms are enormous and really comfortable...,25,3,8.8,"[' Leisure trip ', ' Group ', ' Standard Tripl...",717 day,48.203745,16.335677


In [30]:
df= df[0:5000] 

In [31]:
len(df)

5000

In [32]:
df_hotels = df[['Hotel_Address','Negative_Review', 'Positive_Review', 'Average_Score']]
df_hotels

,Hotel_Address,Negative_Review,Positive_Review,Average_Score
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,I am so angry that i made this post available...,Only the park outside of the hotel was beauti...,7.7
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,No Negative,No real complaints the hotel was great great ...,7.7
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,Rooms are nice but for elderly a bit difficul...,Location was good and staff were ok It is cut...,7.7
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,My room was dirty and I was afraid to walk ba...,Great location in nice surroundings the bar a...,7.7
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,You When I booked with your company on line y...,Amazing location and building Romantic setting,7.7
...,...,...,...,...
4995,1 8 Russell Square Camden London WC1B 5BE Unit...,noisy air conditioning 2 hair dryers not work...,location,8.0
4996,1 8 Russell Square Camden London WC1B 5BE Unit...,small room with single bed tho,Lovely hotel,8.0
4997,1 8 Russell Square Camden London WC1B 5BE Unit...,No Negative,The staff were helpful and friendly,8.0
4998,1 8 Russell Square Camden London WC1B 5BE Unit...,No Negative,Location ambiance and facilities,8.0


In [33]:
pd.options.mode.chained_assignment = None 

# s is input text
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    
    return s

df_hotels['Negative_Review']= df_hotels['Negative_Review'].apply(lambda x : normalize_text(x))
df_hotels['Positive_Review']= df_hotels['Positive_Review'].apply(lambda x : normalize_text(x))
df_hotels['Hotel_Address']= df_hotels['Hotel_Address'].apply(lambda x : normalize_text(x))

In [34]:
tokenizer = tiktoken.get_encoding("cl100k_base")
df_hotels['n_tokens'] = df_hotels["Negative_Review"].apply(lambda x: len(tokenizer.encode(x)))
df_hotels['n_tokens'] = df_hotels["Positive_Review"].apply(lambda x: len(tokenizer.encode(x)))
df_hotels['n_tokens'] = df_hotels["Hotel_Address"].apply(lambda x: len(tokenizer.encode(x)))
df_hotels = df_hotels[df_hotels.n_tokens<8192]
len(df_hotels)

5000

In [35]:
df_hotels

,Hotel_Address,Negative_Review,Positive_Review,Average_Score,n_tokens
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam N...,I am so angry that i made this post available ...,Only the park outside of the hotel was beautiful,7.7,15
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam N...,No Negative,No real complaints the hotel was great great l...,7.7,15
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam N...,Rooms are nice but for elderly a bit difficult...,Location was good and staff were ok It is cute...,7.7,15
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam N...,My room was dirty and I was afraid to walk bar...,Great location in nice surroundings the bar an...,7.7,15
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam N...,You When I booked with your company on line yo...,Amazing location and building Romantic setting,7.7,15
...,...,...,...,...,...
4995,1 8 Russell Square Camden London WC1B 5BE Unit...,noisy air conditioning 2 hair dryers not working,location,8.0,15
4996,1 8 Russell Square Camden London WC1B 5BE Unit...,small room with single bed tho,Lovely hotel,8.0,15
4997,1 8 Russell Square Camden London WC1B 5BE Unit...,No Negative,The staff were helpful and friendly,8.0,15
4998,1 8 Russell Square Camden London WC1B 5BE Unit...,No Negative,Location ambiance and facilities,8.0,15


In [36]:
sample_encode = tokenizer.encode(df_hotels.Negative_Review[0]) 
decode = tokenizer.decode_tokens_bytes(sample_encode)
decode

[b'I',
 b' am',
 b' so',
 b' angry',
 b' that',
 b' i',
 b' made',
 b' this',
 b' post',
 b' available',
 b' via',
 b' all',
 b' possible',
 b' sites',
 b' i',
 b' use',
 b' when',
 b' plan',
 b'ing',
 b' my',
 b' trips',
 b' so',
 b' no',
 b' one',
 b' will',
 b' make',
 b' the',
 b' mistake',
 b' of',
 b' booking',
 b' this',
 b' place',
 b' I',
 b' made',
 b' my',
 b' booking',
 b' via',
 b' booking',
 b' com',
 b' We',
 b' stayed',
 b' for',
 b' ',
 b'6',
 b' nights',
 b' in',
 b' this',
 b' hotel',
 b' from',
 b' ',
 b'11',
 b' to',
 b' ',
 b'17',
 b' July',
 b' Upon',
 b' arrival',
 b' we',
 b' were',
 b' placed',
 b' in',
 b' a',
 b' small',
 b' room',
 b' on',
 b' the',
 b' ',
 b'2',
 b'nd',
 b' floor',
 b' of',
 b' the',
 b' hotel',
 b' It',
 b' turned',
 b' out',
 b' that',
 b' this',
 b' was',
 b' not',
 b' the',
 b' room',
 b' we',
 b' booked',
 b' I',
 b' had',
 b' specially',
 b' reserved',
 b' the',
 b' ',
 b'2',
 b' level',
 b' duplex',
 b' room',
 b' so',
 b' that',
 b

In [37]:
len(decode)

410

In [38]:
df_hotels['ada_v2'] = df_hotels['Positive_Review'].apply(lambda x : get_embedding(x, engine = 'text-embedding-ada-002-2')) # engine should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model

In [39]:
df_hotels[20:40
         ]

,Hotel_Address,Negative_Review,Positive_Review,Average_Score,n_tokens,ada_v2
20,s Gravesandestraat 55 Oost 1092 AA Amsterdam N...,Bed was on upper level with a narrow twist sta...,Friendly staff OostPark a few yards away Good ...,7.7,15,"[0.017190411686897278, 0.0028120712377130985, ..."
21,s Gravesandestraat 55 Oost 1092 AA Amsterdam N...,Our room was an overrated disaster room 231 di...,The breakfast was the only positive element of...,7.7,15,"[-0.005051459185779095, 0.007142528425902128, ..."
22,s Gravesandestraat 55 Oost 1092 AA Amsterdam N...,Sadly I cannot say that the rooms are clean en...,The location is good You need 15min to 20min w...,7.7,15,"[0.011765029281377792, 0.02556644007563591, 0...."
23,s Gravesandestraat 55 Oost 1092 AA Amsterdam N...,Transportation was a bit of a pain but on rout...,Bed was extremely comfy and the staff where we...,7.7,15,"[0.006413136143237352, 0.020551111549139023, 0..."
24,s Gravesandestraat 55 Oost 1092 AA Amsterdam N...,Nothing,Lovely hotel with extremely comfortable huge d...,7.7,15,"[0.02573161944746971, -0.005910688545554876, 0..."
25,s Gravesandestraat 55 Oost 1092 AA Amsterdam N...,The bathroom in our room was a black glass box...,Great location in the park near museums and re...,7.7,15,"[-0.005483401007950306, 0.01410017441958189, -..."
26,s Gravesandestraat 55 Oost 1092 AA Amsterdam N...,Nothing at all to do with the Hotel of course ...,The Hotel itself is in a lovely location a 5mi...,7.7,15,"[0.0049764118157327175, 0.021207015961408615, ..."
27,s Gravesandestraat 55 Oost 1092 AA Amsterdam N...,Careful they are still renovating the building...,Great hotel original concept style,7.7,15,"[-0.0032607032917439938, 0.005221199244260788,..."
28,s Gravesandestraat 55 Oost 1092 AA Amsterdam N...,We had 2 different rooms here and both were du...,The hotel itself is beautiful restaurant is ve...,7.7,15,"[-0.004773873370140791, 0.011211071163415909, ..."
29,s Gravesandestraat 55 Oost 1092 AA Amsterdam N...,There is an ongoing construction enlarging the...,The hotel is located in a beautiful old monast...,7.7,15,"[-0.002456169342622161, 0.017583753913640976, ..."


In [40]:
# search through the reviews for a specific product
def search_docs(df, user_query, top_n=3, to_print=True):
    embedding = get_embedding(
        user_query,
        engine="text-embedding-ada-002-2" 
    )
    df["similarities"] = df['ada_v2'].apply(lambda x: cosine_similarity(x, embedding))

    res = (
        df.sort_values("similarities", ascending=False)
        .head(top_n)
    )
    if to_print:
        display(res)
    return res


res = search_docs(df_hotels, "I want to stay in a hotel where breakfast is the best", top_n=4)

,Hotel_Address,Negative_Review,Positive_Review,Average_Score,n_tokens,ada_v2,similarities
132,s Gravesandestraat 55 Oost 1092 AA Amsterdam N...,It would have been nice if the one responsible...,Good breakfast nice beds nice to live at a hot...,7.7,15,"[0.008635422214865685, 0.00854242593050003, 0....",0.881915
4692,1 8 Russell Square Camden London WC1B 5BE Unit...,We had 3 rooms in total one was a little noisy...,Good breakfast and the decor of the hotel is w...,8.0,15,"[0.004365593194961548, 0.006744793616235256, 0...",0.880628
3634,1 3 Queens Garden Westminster Borough London W...,No Negative,The hotel facilities are good and i enjoy thei...,7.7,15,"[0.0016557652270421386, 0.009121965616941452, ...",0.880539
259,s Gravesandestraat 55 Oost 1092 AA Amsterdam N...,Room design resulted in limited storage space ...,Excellent Breakfasts,7.7,15,"[0.0007433082791976631, 0.007841206155717373, ...",0.877161


In [47]:
res[0:1]['Hotel_Address']

132    s Gravesandestraat 55 Oost 1092 AA Amsterdam N...
Name: Hotel_Address, dtype: object

In [48]:
res = search_docs(df_hotels, "I want to stay in a hotel where breakfast is the best and the location is good", top_n=4)

,Hotel_Address,Negative_Review,Positive_Review,Average_Score,n_tokens,ada_v2,similarities
4306,1 8 Russell Square Camden London WC1B 5BE Unit...,Overpriced bad nights sleep due to noise from ...,Nice breakfast good location,8.0,15,"[0.003134180558845401, 0.005315182730555534, 0...",0.911084
2733,1 3 Queens Garden Westminster Borough London W...,got a basement room with no natural light Tiny...,good location good breakfast,7.7,15,"[-0.005336749367415905, 0.007927946746349335, ...",0.910969
3665,1 3 Queens Garden Westminster Borough London W...,Bed matress was a little too hard,Good location great breakfast,7.7,15,"[-0.003040354698896408, 0.01167367771267891, 0...",0.908914
477,1 15 Templeton Place Earl s Court Kensington a...,Two years in a row we checked in for 10 days O...,Location location location and breakfast,8.5,20,"[-0.005637835245579481, -0.0015237834304571152...",0.908499


In [49]:
res = search_docs(df_hotels, "I will have a vacation with my 2 children and would like to have good activites for my children", top_n=4)

,Hotel_Address,Negative_Review,Positive_Review,Average_Score,n_tokens,ada_v2,similarities
1453,1 2 Serjeant s Inn Fleet Street City of London...,nothing,Staff went out of their way to make my daughte...,9.2,22,"[0.001751572941429913, 0.0017613766249269247, ...",0.841167
3652,1 3 Queens Garden Westminster Borough London W...,Nil,Good location Excellent staff Good fascilities...,7.7,15,"[-0.005229237023741007, 0.033077966421842575, ...",0.834441
2370,1 3 Queens Garden Westminster Borough London W...,The mattress could have been better didn t hav...,I had a very good stay here with my young chil...,7.7,15,"[0.00628528231754899, 0.03105131722986698, -0....",0.831846
1933,1 2 Serjeant s Inn Fleet Street City of London...,None,Perfect location perfect weekend away,9.2,22,"[-0.005650183651596308, 0.018797416239976883, ...",0.817381
